In [6]:
import pandas as pd 
import sqlite3
from datetime import datetime

In [7]:
db_path = "/Users/yuta/Desktop/climb_proj/instance/climbs.db"

connection = sqlite3.connect(db_path)
cursor = connection.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('users',), ('climbs',), ('score',), ('comments',)]


In [8]:
import os 
print(os.path.abspath(db_path))

/Users/yuta/Desktop/climb_proj/instance/climbs.db


In [9]:
query_climb = 'SELECT * FROM climbs'
climbs_df = pd.read_sql_query(query_climb, connection)
climbs_df 

query_user = 'SELECT * FROM users'
user_df = pd.read_sql_query(query_user, connection)
user_df

query_score = 'SELECT * FROM score'
score_df = pd.read_sql_query(query_score, connection)
score_df

,id,performance_score,activity_score,grade,score_ownwer_id,recorded_at
0,1,98,496,10,3,2024-12-02
1,2,95,279,4,3,2024-12-01
2,3,72,451,9,3,2024-11-30
3,4,98,301,10,3,2024-11-29
4,5,80,445,5,3,2024-11-28
5,6,72,318,10,3,2024-11-27
6,7,75,183,3,3,2024-11-26


In [10]:
# this returns today's climbing activity of user with specified id
def excercise_today(id):
    user_climb_data = climbs_df[climbs_df.upload_author_id==id] # make the value inside upload_author_id dynamic
    user_climb_data['date_logged'] = pd.to_datetime(user_climb_data.date_logged) # convert the value to datetime format for conpatibility
    user_data_today = user_climb_data[user_climb_data['date_logged'].dt.date == datetime.now().date()]
    if user_data_today.empty: # need to implement error handling in case there is no excersize history that day
        return None
    else:
        return user_data_today
    
# climbs_completed = excercise_today(3)[excercise_today(3).completed == True]
# climbs_completed

In [11]:
# this returns max_grade of the user with specified id 
def user_max(id):
    user_detail = user_df[user_df.id == id]
    return int(user_detail.current_grade)

# user_max(3)

In [12]:
# this returns performance score based solely on the attempt number and the grade which is before being refined to the final score 

def performance_score(id):
    max_attempts = 100
    grade = excercise_today(id)[excercise_today(id).completed == True].grade # this ensures that its getting only completed climbs
    user_max_grade = user_max(id)
    user_attempt = excercise_today(id)[excercise_today(id).completed == True].attempt # this ensures that its getting only completed climbs
    intensity_level = grade / user_max_grade
    score = intensity_level * (max_attempts / (user_attempt + max_attempts)) * 100
    # print(score)
    return int(score.mean())

# performance_score(3)

In [13]:
def final_score(id):
    score = performance_score(id)
    user_max_grade = user_max(id)
    climbs_completed = excercise_today(id)[excercise_today(id).completed == True]
    completed_climb_num = climbs_completed.count().grade
    completed_climb_num

    # add or deduct number from perfromnace score based on the number of climbs completed
    if completed_climb_num == 1:
        score -= 7
    elif completed_climb_num == 2:
        score -= 6
    elif completed_climb_num == 3:
        score -= 5
    elif completed_climb_num == 4:
        score += 5
    elif completed_climb_num == 5:
        score += 6
    elif completed_climb_num == 6:
        score += 7
    elif completed_climb_num > 6:
        score += 10

    # give extra score if the user cpmpleted climbs with the grade higher than the grade 2 grades lower than their max grade
    base_hard_climbs = climbs_completed[climbs_completed.grade >= user_max_grade -2].count().id
    score += base_hard_climbs * 5
    score

    # give extra score if the user cpmpleted their max grade
    max_grade = climbs_completed[climbs_completed.grade == user_max_grade].count().id
    score += max_grade * 5

    # give extra score if the user cpmpleted climbs with the grade higher than their max grade
    hard_grade = climbs_completed[climbs_completed.grade > user_max_grade].count().id
    score += hard_grade * 10

    if score > 100:
        score = 100
    return score

print(final_score(3))

80


/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_63000/13646849.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_climb_data['date_logged'] = pd.to_datetime(user_climb_data.date_logged) # convert the value to datetime format for conpatibility
/var/folders/80/hv8gpdgn6tndvgyr9pwntpz40000gn/T/ipykernel_63000/13646849.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_climb_data['date_logged'] = pd.to_datetime(user_climb_data.date_logged) # convert the value to datetime format for conpat

In [14]:
score_df

,id,performance_score,activity_score,grade,score_ownwer_id,recorded_at
0,1,98,496,10,3,2024-12-02
1,2,95,279,4,3,2024-12-01
2,3,72,451,9,3,2024-11-30
3,4,98,301,10,3,2024-11-29
4,5,80,445,5,3,2024-11-28
5,6,72,318,10,3,2024-11-27
6,7,75,183,3,3,2024-11-26
